In [3]:
import os

if os.path.exists("bench") == False:
    os.mkdir("bench")

!wget https://arxiv.org/pdf/1706.03762 -O bench/attention.pdf
!wget https://arxiv.org/pdf/2407.01449 -O bench/colpali.pdf
!wget https://arxiv.org/pdf/2310.06825 -O bench/mistral.pdf


--2024-10-10 14:33:42--  https://arxiv.org/pdf/1706.03762
Resolving arxiv.org (arxiv.org)... 151.101.67.42, 151.101.3.42, 151.101.195.42, ...
Connecting to arxiv.org (arxiv.org)|151.101.67.42|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2215244 (2.1M) [application/pdf]
Saving to: ‘bench/attention.pdf’

bench/attention.pdf 100%[===================>]   2.11M  3.94MB/s    in 0.5s    

2024-10-10 14:33:43 (3.94 MB/s) - ‘bench/attention.pdf’ saved [2215244/2215244]

--2024-10-10 14:33:43--  https://arxiv.org/pdf/2407.01449
Resolving arxiv.org (arxiv.org)... 151.101.131.42, 151.101.67.42, 151.101.3.42, ...
Connecting to arxiv.org (arxiv.org)|151.101.131.42|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9380119 (8.9M) [application/pdf]
Saving to: ‘bench/colpali.pdf’

bench/colpali.pdf   100%[===================>]   8.95M  5.41MB/s    in 1.7s    

2024-10-10 14:33:45 (5.41 MB/s) - ‘bench/colpali.pdf’ saved [9380119/9380119]

--2024-10-10 

In [4]:
# from sentence_transformers import SentenceTransformer
# import pymupdf
# from semantic_text_splitter import TextSplitter
# import time
# splitter = TextSplitter(1000)
# files = []
# model = SentenceTransformer('BAAI/bge-small-en-v1.5')

# # get all pdfs from test_files
# now = time.time()

# for file in os.listdir("bench"):

#   text = []
#   doc = pymupdf.open("bench/" + file)

#   for page in doc:
#       text.append(page.get_text())

#   text = " ".join(text)
#   chunks = splitter.chunks(text)
#   embeddings = model.encode(chunks)


# print(time.time() - now)



In [5]:
import torch
import gc
gc.collect()
torch.cuda.empty_cache()

In [15]:
from fastembed import TextEmbedding
import pymupdf
from semantic_text_splitter import TextSplitter
import time
import os
splitter = TextSplitter(1000)
files = []
model = TextEmbedding(model_name = "BAAI/bge-small-en-v1.5",
                      providers=["CUDAExecutionProvider", "CPUExecutionProvider"])

# get all pdfs from test_files
now = time.time()

for file in os.listdir("bench"):

  text = []
  print(file)
  doc = pymupdf.open("bench/" + file)

  for page in doc:
      text.append(page.get_text())

  text = " ".join(text)
  chunks = splitter.chunks(text)
  embeddings = list(model.embed(chunks))
  
print(time.time() - now)


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

colpali.pdf
mistral.pdf
attention.pdf
0.2967040538787842


In [1]:
from embed_anything import EmbeddingModel, WhichModel, TextEmbedConfig, embed_query, embed_directory
import time

import pymupdf
from semantic_text_splitter import TextSplitter
import time
import os

config = TextEmbedConfig(256, 2)
model = EmbeddingModel.from_pretrained_onnx(WhichModel.Bert, "BAAI/bge-small-en-v1.5")
splitter = TextSplitter(1000)
files = []


# get all pdfs from test_files
now = time.time()

for file in os.listdir("bench"):
  text = []
  print(file)
  doc = pymupdf.open("bench/" + file)

  for page in doc:
      text.append(page.get_text())
  
  text = " ".join(text)
  chunks = splitter.chunks(text)
  embeddings = embed_query(chunks, model, config)
print(time.time() - now)
